Find Incidents


In [1]:
import json
import requests
from pymilvus import model
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction

/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from pymilvus import MilvusClient

DIMENSION = 384 
MILVUS_URL = "http://192.168.1.163:19530" 
COLLECTION_NAME = "nyccollisions"

# -----------------------------------------------------------------------------
# Connect to Milvus

# Milvus Lite
# milvus_client = MilvusClient(uri="streetcams.db")

# Local Docker Server
milvus_client = MilvusClient( uri=MILVUS_URL )

In [6]:
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import pprint

model = SentenceTransformerEmbeddingFunction('all-MiniLM-L6-v2',device='cpu' )

earlyincident = "Crash occurred on JACKSON STREET with off street CHERRY STREET MANHATTAN, NY 10002 with lat/long 40.712208 -73.98072 at 0:02 on 2024-04-30T00:00:00.000 with vehicles Taxi and Bus including 0 injuries"


# Define search parameters
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 16}
}

# Use first record as search record
query_vector = [model(earlyincident)]

# Execute the search on the 'vector' field
search_results = milvus_client.search(
    COLLECTION_NAME,
    data=query_vector,
    filter='crash["vehicle_type_code1"] in ["Taxi"]',
    output_fields=["id", "crash"],
    search_params=search_params,
    limit=20
)

# Print search results
for hits in search_results:
    for hit in hits:
        print(f"Hit: {hit}")


/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Hit: {'id': 4720812, 'distance': 0.0, 'entity': {'id': 4720812, 'crash': {'details': 'Crash occurred on JACKSON STREET with off street CHERRY STREET MANHATTAN, NY 10002 with lat/long 40.712208 -73.98072 at 0:02 on 2024-04-30T00:00:00.000 with vehicles Taxi and Bus including 0 injuries', 'on_street_name': 'JACKSON STREET', 'off_street_name': 'CHERRY STREET', 'crash_date': '2024-04-30T00:00:00.000', 'crash_time': '0:02', 'borough': 'MANHATTAN', 'zip_code': '10002', 'latitude': '40.712208', 'longitude': '-73.98072', 'location': 'JACKSON STREET MANHATTAN, NY 10002', 'number_of_persons_injured': '0', 'number_of_persons_killed': '0', 'number_of_pedestrians_injured': '0', 'number_of_pedestrians_killed': '0', 'number_of_cyclist_injured': '0', 'number_of_cyclist_killed': '0', 'number_of_motorist_injured': '0', 'number_of_motorist_killed': '0', 'contributing_factor_vehicle_1': 'Failure to Yield Right-of-Way', 'vehicle_type_code1': 'Taxi', 'contributing_factor_vehicle_2': 'Unspecified', 'vehicle_

In [9]:
# Execute the search on the 'vector' field
search_results = milvus_client.search(
    COLLECTION_NAME,
    data=query_vector,
    filter='crash["borough"] like "MANHATTAN" && crash["vehicle_type_code1"] in ["Taxi", "Sedan", "Bus"]',
    output_fields=["id", "crash"],
    search_params=search_params,
    limit=5
)

# Print search results
for hits in search_results:
    for hit in hits:
        print(f"Hit: {hit}")

Hit: {'id': 4720812, 'distance': 0.0, 'entity': {'id': 4720812, 'crash': {'details': 'Crash occurred on JACKSON STREET with off street CHERRY STREET MANHATTAN, NY 10002 with lat/long 40.712208 -73.98072 at 0:02 on 2024-04-30T00:00:00.000 with vehicles Taxi and Bus including 0 injuries', 'on_street_name': 'JACKSON STREET', 'off_street_name': 'CHERRY STREET', 'crash_date': '2024-04-30T00:00:00.000', 'crash_time': '0:02', 'borough': 'MANHATTAN', 'zip_code': '10002', 'latitude': '40.712208', 'longitude': '-73.98072', 'location': 'JACKSON STREET MANHATTAN, NY 10002', 'number_of_persons_injured': '0', 'number_of_persons_killed': '0', 'number_of_pedestrians_injured': '0', 'number_of_pedestrians_killed': '0', 'number_of_cyclist_injured': '0', 'number_of_cyclist_killed': '0', 'number_of_motorist_injured': '0', 'number_of_motorist_killed': '0', 'contributing_factor_vehicle_1': 'Failure to Yield Right-of-Way', 'vehicle_type_code1': 'Taxi', 'contributing_factor_vehicle_2': 'Unspecified', 'vehicle_